In [77]:
from dotenv import load_dotenv,dotenv_values
load_dotenv()
from PIL import Image
from IPython.display import display
from pydantic import BaseModel, Field
import pandas as pd 
import anthropic
import base64
import os 
from sudulunu.helpers import pp, dumper

claudy = dotenv_values(".env")['claudy']

anthro_client = anthropic.Anthropic(
     api_key=claudy,
)

In [78]:


class value(BaseModel):
    shade_name: str = Field(description="The common name for the colour value")
    hex_code : str= Field(description="Hex code for the colour value")
    value_share : float = Field(description="The share of the image that is this colour value")

class values(BaseModel):
    records: list[value]

tools = [
        {
            "name": "value_extract",
            "description": "extract colour value data",
            "input_schema": values.model_json_schema()
        }
    ]

def get_hexes(image_path, prompt):

    with open(image_path, "rb") as img:
        stringo = base64.b64encode(img.read()).decode("utf-8")

    if ".png" in image_path.lower():
         media_type = 'image/png'
    else:
        media_type = "image/jpeg"

    message = anthro_client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=1024,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": media_type,
                            "data": stringo,
                        },
                    },
                    {
                        "type": "text",
                        "text": prompt
                    }
                ],
            }
        ],
        tools=tools,
        tool_choice={"type": "tool", "name": "value_extract"}
    )
    # print(message)

    records = message.content[0].input['records']
    frame = pd.DataFrame.from_records(records)
    # img = Image.open(image_path)
    # display(img)
    # print(frame)
    # print("Sum: ", frame['value_share'].sum())
    # print(frame.columns.tolist())

    return frame


In [79]:
# pathos = '/Users/josh/Github/site/python/image_archive'

# prompto = """Analyise this image and extract all the colour values as hex codes. 
# Return the data as a python dictionary with the hex code as the key and the 
# propotion of the image's pixels that is that hex code. Return only the dictionary and no other words. Return at least 10 colour values"""

# to_do = os.listdir(pathos)

# include = ['.jpg', '.jpeg', 'png']

# fillos = os.listdir(pathos)
# fillos = [x for x in fillos if any(endo in x.lower() for endo in include)]

# records = []

# counter = 0
# for fillo in fillos:
#     counter += 1
#     if counter % 50 == 0:
#         print(counter)

#     old = pd.read_csv('/Users/josh/Github/site/python/tests/colour_filter/output/colours.csv')
#     already_done = old['img_path'].unique().tolist()

#     if fillo not in already_done:
#         record = get_hexes('/Users/josh/Github/site/python/tests/colour_filter/input/03598547cb.jpg', prompto)
#         record['img_path'] = fillo
#         # print(record)
#         records.append(record)

#         new = pd.concat(records)
#         frame = pd.concat([new, old])
#         frame.drop_duplicates(subset=['shade_name','hex_code','value_share','img_path'], inplace=True)

#         dumper('/Users/josh/Github/site/python/tests/colour_filter/output', 'colours', frame)

        


50
100
150
200
250
300


In [87]:
### Need to connect  output as a json

colours = pd.read_csv('/Users/josh/Github/site/python/tests/colour_filter/output/colours.csv')
# ['shade_name', 'hex_code', 'value_share', 'img_path']

tog = pd.read_csv('/Users/josh/Github/site/python/scrap/together.csv')
# ['Date', 'Title', 'img_path', 'Caption', 'Colours', 'Style', 'Subject',
#   'Keywords', 'Category', 'img_alt', 'Width', 'Height']
tog = tog[['Date','img_path',]]

combo = pd.merge(colours, tog, on='img_path')

records = []

for index, row in combo.iterrows():
    record = {"date": row['Date'], "colour" :row['hex_code'], "share": row['value_share'] * 100}
    records.append(record)

cat = pd.DataFrame.from_records(records)
pp(cat)

with open('/Users/josh/Github/site/python/tests/colour_filter/output/colours.json', 'w') as f:
    cat.to_json(f, orient='records')

            date   colour  share
0     2020-10-31  #E5E5E5   25.0
1     2020-10-31  #FF9999   15.0
2     2020-10-31  #89A894   12.0
3     2020-10-31  #8B6D5C   10.0
4     2020-10-31  #A9A9A9    8.0
...          ...      ...    ...
3321  2022-08-23  #FF7B7B    8.0
3322  2022-08-23  #F5F5F5    7.0
3323  2022-08-23  #966F33    6.0
3324  2022-08-23  #36454F    5.0
3325  2022-08-23  #98FF98    4.0

[3326 rows x 3 columns]
['date', 'colour', 'share']
